# AEMET OpenData
Aemet pone a disposición de los interesados una interfaz amigable para hacer la descarga de todos los datos que oferta en OpenData (https://opendata.aemet.es/centrodedescargas/inicio).

En este notebook se muestra cómo se puede acceder a un subconjunto de los datos ofertados utilizando Python, bien de modo interactivo o utilizando el __módulo aemet_open_data__. Para trabajar en Python tienes que haber instalado la librería Pandas.

Los datos que puedes decargar utilizando este notebook son:
1. Estaciones climatológicas con datos en Aemet OpenData. Aemet tiene muchas más estaciones con datos, pero la mayor parte no están en OpenData.
1. Variables climatológicas diarias.
1. Variables climatológicas mensuales.

Desde Aemet puedes descargar las variables climatológicas, pero se ponen limitaciones en el número de datos a descargar por petición. Por ejemplo, para los datos diarios hay que hacer una petición por estación y un máximo de 5 años de serie; para los datos mensuales, una estación y un máximo de 36 meses. Si quieres obtener datos de más de una estación para una serie larga tienes que repetir el proceso. En estos casos es cuando utilizar el módulo aemet_open_data u otro similar te permite ahorrar tiempo.

Un cierto inconveniente es que entre los datos de las estaciones climatológicas OpenData no figura el primer dato de cada serie. Si quieres obtener las series completas tendrás que empezar haciendo peticiones desde una fecha inicial antigua e ir moviéndo progresivamente el rango hasta encontrar el inicio, lo que es tedioso si no utilizas un proceso automatizado, además de consumidor de recursos, pues en ocasiones tienes que hacer muchas peticiones que no van a devolver datos.  

Si no estás interesado en el modo interactivo y lo que quieres es disponer rápido de los datos, te recomiendo ir directamente al apartado de cómo se utiliza el módulo aemet_open_data.

En todos los casos necesitas una __clave de acceso__. Entra en la página de Aemet https://opendata.aemet.es/centrodedescargas/inicio y solicita una clave para operar en el servicio (APIKEY). La clave es una serie alfanumérica muy larga, sálvala en el fichero apikey.txt. Tu apikey no caduca.

## 1.- USAR Python PASO A PASO

Si tienes curiosidad de ver como se funciona la descarga de datos en Python instrucción por instrucción.

Pasos:
1. Se hace una petición de datos a Aemet, para lo que se necesita formar una url específica de cada tipo de datos que queremos descargar.
1. Si la petición tiene éxito devuelve 2 url: una para descargar los datos y otra para descargar los metadatos. Estas 2 url tienen una tiempo de validez.


In [18]:
# Lo primero tengo que importar las librerías que voy a utilizar
# recuerda que tienes que instalar Pandas
from datetime import date
import json
import pandas as pd
import requests

In [3]:
# Leo mi apikey
with open('apikey.txt') as f:
    myapikey = f.readline()

# Leo las urls que tengo preparadas (hay muchas más)
with open('urls.json', 'r') as f:
    urls = json.load(f)

querystring = {"api_key":myapikey}
headers = {'cache-control': "no-cache"}

# ver la url que teng preparadas y los parámetros (esta petición no tiene)aemet_open_data
urls.keys()

dict_keys(['estaciones', 'estacion_dia', 'estacion_mes'])

Es decir, en el fichero urls.json tengo guardadas 3 urls para descargar datos:
1. estaciones. Los datos de las estaciones
2. estacion_dia. Los datos de las variables climatológicas diarias por estación
3. estacion_mes. Los datos de las variables climatológicas mensuales por estación 

### Estaciones AEMET OpenData
Petición de todas las estaciones meteorológicas que sirven datos en el servicio OpenData 

In [10]:
# Para ver la información completa para descargar los datos de las estaciones
urls['estaciones']

['https://opendata.aemet.es/opendata/api/valores/climatologicos/inventarioestaciones/todasestaciones',
 []]

In [11]:
# veo que sólo necesito la url, es decir: 
urls['estaciones'][0]

'https://opendata.aemet.es/opendata/api/valores/climatologicos/inventarioestaciones/todasestaciones'

In [12]:
# aquí hago la petición (request) y veo la respuesta (response y content)
url = urls['estaciones'][0]
response = requests.request("GET", url, headers=headers, params=querystring)
content = json.loads(response.content)
content

{'descripcion': 'exito',
 'estado': 200,
 'datos': 'https://opendata.aemet.es/opendata/sh/5a60779b',
 'metadatos': 'https://opendata.aemet.es/opendata/sh/0556af7a'}

La petición me ha devuelto la respuesta del servidor de Aemet.

* Un código de la respuesta (estado) y una descripción del código
* Dos urls; una para los metadatos y otra para los datos. Estas urls están activas durante un tiempo limitado

In [13]:
# si la petición tiene éxito puedo descargar los datos; el formato original es json
# pero para majorar la visibilidad lo almaceno en una dataframe de pandas
df = pd.read_json(content["datos"], encoding='latin-1')
df.head()

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud
0,413515N,BARCELONA,74,0252D,ARENYS DE MAR,08186,023224E
1,411734N,BARCELONA,4,0076,BARCELONA AEROPUERTO,08181,020412E
2,412506N,BARCELONA,408,0200E,"BARCELONA, FABRA",,020727E
3,412326N,BARCELONA,6,0201D,BARCELONA,08180,021200E
4,414312N,BARCELONA,291,0149X,MANRESA,08174,015025E


In [14]:
# Si la petición tuvo éxito también puedo descargar los metadatos
# Observa que algunas columnas se visializan truncadas, pero si se graba
# en un fichero se puede ver el texto completo
df_meta = pd.read_json(content["metadatos"], encoding='latin-1')
df_meta.head()

,unidad_generadora,periodicidad,descripcion,formato,copyright,notaLegal,campos
0,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'latitud', 'descripcion': 'latitud de l..."
1,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'provincia', 'descripcion': 'provincia ..."
2,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'indicativo', 'descripcion': 'indicativ..."
3,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'altitud', 'descripcion': 'altitud de l..."
4,Servicio del Banco de Datos Nacional de Climat...,1 vez al día,Inventario de estaciones para el apartado Valo...,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'nombre', 'descripcion': 'ubicación de ..."


In [15]:
# Los datos en df y en df_meta los puedo salvar en un fichero con varios formatos
# aquí utilizo el formato csv, que puede ser leido con Excel o Calc, por ejemplo
# para guardar los datos (df) y sus metadatos (df_meta) cambia el nombre al fichero en dst 
# y en la instrucción to_csv pon df o df_meta según el caso
dst = '.\download\estaciones.csv'
df.to_csv(dst, index=False)

### Datos meteorológicos diarios de una estación y un rango de fechas de hasta 5 años
Si quieres más de una estación o un periodo de más de 5 años hay que repetir el proceso para cada estación y periodo de hasta 5 años

In [16]:
# vemos la url a la que se va a hacer la petición y los parámetros:
# 2 fechas en un formato específico y un código de estación idema
urls['estacion_dia']

['https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/fechaIniStr/fechafin/fechaFinStr/estacion/idema',
 ['fechaIniStr format AAAA-MM-DDTHH:MM:SSUTC',
  'fechaFinStr format AAAA-MM-DDTHH:MM:SSUTC',
  'idema']]

La respuesta tiene la fefinición de la url y 3 parámetros que tengo que sustituir por un valor concreto en la petición:
* fechaIniStr y fechaFinStr, fechas iniciales de la petición de datos con formato AAAA-MM-DDTHH:MM:SSUTC
* idema, código de la estación

In [17]:
# Doy valor a una url que tengo preformada (url), defino los nombres de los parámetros (params)
# y asigno los valores de los parámetros
# Sólo puedo pedir los datos de una estación en un rango de fechas de menos de 5 años
# Si quiero más estaciones o un rango de fehas mayor hay que repetir el proceso cambiando
# el contenido de values
url = urls['estacion_dia'][0]
params = ('fechaIniStr', 'fechaFinStr', 'idema') 
values = ('2022-01-01T00:00:00UTC', '2023-01-01T23:59:59UTC', '7178I')

for par, value in zip(params, values):
    url = url.replace(par, value) 

print(url)

response = requests.request("GET", url, headers=headers, params=querystring)
response_json = response.json()
response_json

https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2022-01-01T00:00:00UTC/fechafin/2023-01-01T23:59:59UTC/estacion/7178I


{'descripcion': 'exito',
 'estado': 200,
 'datos': 'https://opendata.aemet.es/opendata/sh/d73b5665',
 'metadatos': 'https://opendata.aemet.es/opendata/sh/b3aa9d28'}

In [18]:
# Igual que antes: si la respuesta tiene éxito puedo descargar los datos
df = pd.read_json(response_json["datos"], encoding='latin-1')
df.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2022-01-01,7178I,MURCIA,MURCIA,62,"13,2","0,0","2,9",07:40,"23,5",15:10,30.0,"0,8","3,9",08:10,"8,6","1022,7",Varias,"1019,2",15
1,2022-01-02,7178I,MURCIA,MURCIA,62,"13,0","0,0","3,5",07:10,"22,6",15:20,30.0,"1,1","3,6",22:20,"8,0","1022,7",10,"1018,9",15
2,2022-01-03,7178I,MURCIA,MURCIA,62,"12,2","0,0","2,1",07:10,"22,4",15:40,23.0,"0,8","4,2",19:30,"8,7","1020,5",00,"1013,9",24
3,2022-01-04,7178I,MURCIA,MURCIA,62,"13,8","0,0","4,3",04:10,"23,3",14:30,29.0,"1,4","10,8",20:30,"8,5","1013,9",00,"1002,5",16
4,2022-01-05,7178I,MURCIA,MURCIA,62,"13,2","0,0","10,1",23:59,"16,2",14:30,28.0,"4,7","15,8",02:20,"5,5","1011,8",10,"1006,6",02


In [19]:
# y los metadatos
df_meta = pd.read_json(response_json["metadatos"], encoding='latin-1')
df_meta.head()

,unidad_generadora,periodicidad,descripcion,formato,copyright,notaLegal,campos
0,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'fecha', 'descripcion': 'fecha del dia ..."
1,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'indicativo', 'descripcion': 'indicativ..."
2,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'nombre', 'descripcion': 'nombre (ubica..."
3,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'provincia', 'descripcion': 'provincia ..."
4,Servicio del Banco Nacional de Datos Climatoló...,"1 vez al día, con un retardo de 4 días",Climatologías diarias,application/json,© AEMET. Autorizado el uso de la información y...,https://www.aemet.es/es/nota_legal,"{'id': 'altitud', 'descripcion': 'altitud de l..."


In [20]:
# y salvar los datos y/o los metadatos a fichero en formato csv
fo = '.\download\murcia_dia.csv'
df.to_csv(fo, index=False)

## 2.- TUTORIAL DE USO DEL MÓDULO aemet_open_data
El módulo tiene utilidad sobre todo en los siguientes casos:
* Necesitamos pedir datos de varias estaciones
* El periodo de datos que necesitamos excede el establecido por Aemet para una petición

Es decir, cuando necesitamos datos de una o más estaciones y en periodos de más de 5 años para los datos diarios o 36 meses para datos mensuales.

En estos casos su utilización nos permite no tener que escribir varias veces la url con las peticiones y salvar cada una a un fichero; el módulo lo hace por nosotros.

Lo primero que hay que hacer son las importaciones (recuerda que tienes que haber instalado Pandas):

In [1]:
# Lo más simple es tener el notebook en el directorio donde has guardado el fichero aemet_open_data.py
# hay otras técnnicas que no se consideran aquí
from datetime import date
import inspect
import pandas as pd
from pathlib import Path

from aemet_open_data import AemetOpenData

# instanciación de un objeto AemetOpenData
# el objeto lee la clave de acceso del fichero apikey.txt
# que está situado en el mismo directorio que el notebook y aemet_open_data.py
aod = AemetOpenData()

A continuación vamos a ver las funciones que podemos utilizar en aod:

In [2]:
aod.get_public_methods()

['concatenate_files',
 'daily_ranges_get',
 'data_meteo_stations',
 'directory_exists',
 'file_exists',
 'file_names_in_dir',
 'get_public_methods',
 'meteo_stations',
 'years_ranges_get']

De estas solo vamos a utilizar unas pocas, que son las que utilizamos para descargar los datos
### Características de las estaciones en OpenData
La función que nos permite descargar ests datos se llama __meteo_stations()__. Vamos a ver la documentación de la función que nos permite descargar las características de las estaciones meteorológicas:

In [3]:
# Documentación de la función que vamos a utilizar meteo_stations()
print(aod.meteo_stations.__doc__)


        Retrieves characteristics of meteorological stations in Aemet
            OpenData and saves them to a file

        Parameters
        ----------
        dir_out: The name of the directory where the data file will be saved
        metadata: If True only metadata will be saved, otherwise only data
        will be saved

        Returns
        -------
        file name with data or metadata
        


In [4]:
# Parámetros de la función meteo_stations()
parameters = inspect.signature(aod.meteo_stations).parameters
print("Parámetros de la función:")
for name, param in parameters.items():
    print(f"{name}: {param.annotation}")

Parámetros de la función:
dir_out: <class 'str'>
metadata: <class 'bool'>


Vamos a dar __valor a los parámetros y a ejecutar meteo_stations()__:

In [5]:
# en dir_out puedes poner cualquier nombre de directorio con permisos de escritura
# si trabajas en Windows, pon una r delante, por ej. r'C:\Users\user1\Documents\tmp'
# si no pones la r delante tienes que escribirlo así: 'C:\\Users\\user1\\Documents\\tmp'  
dir_output = './download'

# si metadata es True descarga los metadatos; si es False descarga los datos 
# prueba a asignar a la variable metadata los valores True o False
metadata = False

# al hacer la llamada a la función, metadata lo escribimos como metadata = metadata
# esto es debido a que el parámetro metadata es opcional y tiene como valor por defecto
# True; vamos a escribir siempre todos los parámetros 
file_name = aod.meteo_stations(dir_output, metadata = metadata)

Downloaded data in estaciones_open_data.csv


Si ahora quieres descargar los metadatos, vuelve a ejecutar la celda anterior cambiando metadata con el valor True

In [6]:
# Para ver los datos descargados puedes utilizar un programa que abra ficheros csv,
# por ejemplo un editor de texto plano, Excel, Calc u otros.
# Desde este notebook podemos utilizar la siguiente instrucción
f = Path(dir_output).joinpath(file_name)
df = pd.read_csv(f)
# vemos las 5 primeras líneas
df.head()
# para ver las 5 últimas escribimos df.tail()
# se puede ver el fichero entero, pero no nos distraemos con esto

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud
0,413515N,BARCELONA,74,0252D,ARENYS DE MAR,8186.0,023224E
1,411734N,BARCELONA,4,0076,BARCELONA AEROPUERTO,8181.0,020412E
2,412506N,BARCELONA,408,0200E,"BARCELONA, FABRA",NaN,020727E
3,412326N,BARCELONA,6,0201D,BARCELONA,8180.0,021200E
4,414312N,BARCELONA,291,0149X,MANRESA,8174.0,015025E


### Datos climatológicos diarios o mensuales
La descarga de los datos climatológicos diarios o mensuales es lo que principalmente justifica la utilitilización del módulo. Se realiza llamando a la función __data_meteo_stations()__

Al igual que la función anterior, vamos a ver la documentación de la función y los parámetros que utiliza

In [7]:
# Documentación de la función que vamos a utilizar data_meteo_stations()
print(aod.data_meteo_stations.__doc__)


        Retrieves daily or monthly climatological data from Aemet OpenData
            server.

        Parameters
        ----------
        var_type : day for daily variables, month for monthly ones
        d1 : Initial date (daily data) or year (monthly data)
        d2 : Final date or year. Equal d1
        estaciones : List of stations or only one station as str
        dir_out : Each request to Aemet server is saved as a csv file in the
            directory dir_out        
        metadata: If True only metadata are downloaded, otherwise only data
            are downloaded
        verbose: If True shows detailed information of results request in the
            screen
        use_files : If True checks that the file name already exists in dir_out
            and does not make the request to the server; otherwise the request
            is made and the pre-existing file is overwritten.
        Returns
        -------
        [] with the the names of saved csv files. Each file h

In [8]:
# Parámetros de data_meteo_stations()
parameters = inspect.signature(aod.data_meteo_stations).parameters
print("Parámetros de la función:")
for name, param in parameters.items():
    print(f"{name}: {param.annotation}")

Parámetros de la función:
var_type: <class 'str'>
d1: <class 'datetime.date'>
d2: <class 'datetime.date'>
estaciones: typing.Union[tuple[str], list[str], str]
dir_out: <class 'str'>
metadata: <class 'bool'>
verbose: <class 'bool'>
use_files: <class 'bool'>


A continuación vamos a __asignar valores a los parámetros__:

In [2]:
# Puede valer 'day' para descargar datos diarios o 'month' para datos mensuales
var_type = 'day'

# A continuación definimos el rango de fechas en que queremos descaragr los datos
# lo hacemos definiendo 2 fechas, una incicial y otra final
# estas fechas las construimos con la función date(AAAA, MM, DD), en la que
# (AAAA, MM, DD) representan el (año, mes, día) como números
d1 = date(2000, 1, 1)
d2 = date(2023, 1, 1)

# Listado de estaciones; el código de la estación lo tomamos de las características
# de las estaciones que ya hemos descargado
estaciones = ('7178I', '7002Y')

# Ruta al directorio donde vamos a descargar los datos
# (recuerda las observaciones sobre cómo se escriben los separadores de directorios
# en Windows que hemos visto anteriormente)
dir_output = './download'

# si metadata es True descarga los metadatos; si es False descarga los datos 
metadata = False

# Si True aparece una descripción más completa en pantalla de cada petición que si
# es False
verbose = True

# El programa salva los resultados de cada petición al servidor en un fichero.
# Si para descargar los datos que necesitamos es necesario realizar muchas peticiones
# al servidor y el proceso se interrumpe, al iniciar el proceso otra vez, la descarga
# no se realiza si los datos ya se han descargado (use_files = True); mientras si
# use_files = False, los datos se vuelven a descargar y se sobreescriben los ficheros
# existentes
use_files = True

# Para que use_files = True proceda como se espera no podemos cambiar el intervalo temporal
# de la petición, pues si lo hacemos los datos se descargarán con otros nombres de ficheros
# y la utilidad del parámetro se perderá al no encontrar ficheros descargados con el 
# mismo nombre

Y a continuación __ejecutamos data_meteo_stations()__:

In [11]:
# En la llamada a la función hay algunos parámetros que se asignan en la forma
# nombre de parámetro = variable (en este caso se han hecho coincidir); son parámetros
# opcionales que tienen valores por defecto y en lo que no nos extendemos para no alargar.
file_names = aod.data_meteo_stations(var_type, d1, d2, estaciones, dir_output, 
                                     metadata = metadata, verbose = verbose,
                                     use_files = use_files)
print('\nFicheros descargados')
print(file_names)

7178I, 2000-01-01T00:00:00UTC, 2004-12-30T23:59:59UTC: 200, OK exito
7178I, 2004-12-31T00:00:00UTC, 2009-12-30T23:59:59UTC: 200, OK exito
7178I, 2009-12-31T00:00:00UTC, 2014-12-30T23:59:59UTC: 200, OK exito
7178I, 2014-12-31T00:00:00UTC, 2019-12-30T23:59:59UTC: 200, OK exito
7178I, 2019-12-31T00:00:00UTC, 2023-01-01T23:59:59UTC: 200, OK exito
7002Y, 2000-01-01T00:00:00UTC, 2004-12-30T23:59:59UTC: 200, OK No hay datos que satisfagan esos criterios
7002Y, 2004-12-31T00:00:00UTC, 2009-12-30T23:59:59UTC: 200, OK exito
7002Y, 2009-12-31T00:00:00UTC, 2014-12-30T23:59:59UTC: 200, OK exito
7002Y, 2014-12-31T00:00:00UTC, 2019-12-30T23:59:59UTC: 200, OK exito
7002Y, 2019-12-31T00:00:00UTC, 2023-01-01T23:59:59UTC: 200, OK exito

Ficheros descargados
['7178I_20000101T000000UTC_20041230T235959UTC.csv', '7178I_20041231T000000UTC_20091230T235959UTC.csv', '7178I_20091231T000000UTC_20141230T235959UTC.csv', '7178I_20141231T000000UTC_20191230T235959UTC.csv', '7178I_20191231T000000UTC_20230101T235959UTC.c

El resultado en el ejemplo muestra que todas las peticiones internas al servidor de datos. Hay una petición que no encuentra datos; en este caso es debido a que en el rango de fechas de la primera petición la estación no tiene datos.<br>
Si a continuación vuelves a ejecutar data_meteo_stations(), verás que los ficheros ya descargados no se vuelven a descargar (hasta que no los eliminer de dir_output).<br>  
Tomando como base este ejemplo, puedes volver a la celda donde se dan valor a los parámetros y a continuación ejecutar otra vez data_meteo_stations() con tus valores. 

Para ver los ficheros desde este notebook usamos Pandas (o desde fuera Excel, Calc, etc.).

In [13]:
# Para ver uno de los ficheros
f_ver = '7178I_20041231T000000UTC_20091230T235959UTC.csv'
f = Path(dir_output).joinpath(f_ver)
df = pd.read_csv(f, decimal=',')
# vemos las 5 primeras líneas
df.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin
0,2004-12-31,7178I,MURCIA,MURCIA,62,9.4,"0,0",0.7,05:00,18.0,15:10,28.0,1.1,4.2,21:35,9.1,1024.8,00,1021.6,15
1,2005-01-01,7178I,MURCIA,MURCIA,62,10.6,"0,0",2.6,06:40,18.6,15:30,25.0,1.4,5.0,10:45,9.2,1024.3,24,1021.6,Varias
2,2005-01-02,7178I,MURCIA,MURCIA,62,11.5,"0,0",3.4,05:00,19.6,14:15,5.0,1.9,6.4,16:19,7.8,1026.3,24,1023.6,15
3,2005-01-03,7178I,MURCIA,MURCIA,62,8.6,"2,9",2.2,06:30,15.0,16:10,4.0,1.1,5.0,13:40,3.0,1030.3,21,1026.3,Varias
4,2005-01-04,7178I,MURCIA,MURCIA,62,11.2,"0,0",6.2,23:59,16.2,14:00,31.0,0.8,4.7,03:02,7.2,1031.6,11,1028.4,16


Si te fijas en la instrucción read_csv, hemos incluido el parámetro decimal con valor ','. Esto es debido a que los datos con números reales, Aemet los sirve con separador de decimal ','. El visualizador que hemos utilizado trabaja con números reales con separador '.' y con esta indicación entiende entonces los datos como números reales y los representa con el separador '.'.<br>
Sin embargo, observarás que la columna __prec__ no se ha realizado esta conversión; esto es debido que hay valores que no pueden ser convertidos a números porque incluyen caracteres no numéricos que deben ser adaptados para poder tratar la columna como numérica. <br>
En resumen, __tenemos que posprocesar los datos descargados__.


### Unir los ficheros descargados en uno solo
En ocasiones, el número de ficheros descargados es elevado. En vez de ir abriéndolos y copiando el contenido en otro, podemos utilizar la función __concatenate_files()__ 

In [14]:
# Documentación de la función que vamos a utilizar concatenate_files()
print(aod.concatenate_files.__doc__)


        Concatenates data csv files in dir_name in a new csv file named
            file_name; data csv files were daily or monthly data files
            saved with the names given in this app; you select the type
            of files to be saved by using var_type

        Parameters
        ----------
        var_type: Day for daily variables, month for monthly ones
        dir_name: Directory path where the files
        files2concat: If len(files2concat) > 0, only concats these files;
            otherwise concats the files that match a regex pattern; this
            pattern differs depending on var_type
        files2exclude: If len(files2concat) == 0 and len(files2exclude) > 0
            the files in files2exclude will not be considered
        ask_overwrite: If file_name exists, ask before overwrite it
        
        Returns
        -------
        List of concatenated csv files
        


In [15]:
# Parámetros de concatenate_files()
parameters = inspect.signature(aod.concatenate_files).parameters
print("Parámetros de la función:")
for name, param in parameters.items():
    print(f"{name}: {param.annotation}")

Parámetros de la función:
var_type: <class 'str'>
dir_name: <class 'str'>
files2concat: [<class 'str'>]
files2exclude: [<class 'str'>]
ask_overwrite: <class 'bool'>


A continuación __asignamos los valores de la función concatenate_files()__:

In [4]:
# Los parámetros var_type and dir_name ya les hemos asignado su valor en una celda 
#  más arriba, por lo que no es necesario definirlos otra vez. No obstante, si vas
#  a ejecutar solo este método o por alguna otra razon quieres cambiarlos desde aquí
#  tienes que quitar el comentario ('#')a las siguientes 2 líneas y darles el valor 
#  deseado
# var_type = 'day'
# dir_output = './download'

files2concat = []
files2exclude = []
ask_overwrite = True

In [7]:
# ahora la ejecutamos
files = aod.concatenate_files(var_type, dir_output, files2concat, files2exclude, ask_overwrite)
print('Concatenated files: ', len(files))
print(files)

File meteo_data_day.csv exists, overwrite? (y/n):  y


Concatenated files:  9
['7002Y_20041231T000000UTC_20091230T235959UTC.csv', '7002Y_20091231T000000UTC_20141230T235959UTC.csv', '7002Y_20141231T000000UTC_20191230T235959UTC.csv', '7002Y_20191231T000000UTC_20230101T235959UTC.csv', '7178I_20000101T000000UTC_20041230T235959UTC.csv', '7178I_20041231T000000UTC_20091230T235959UTC.csv', '7178I_20091231T000000UTC_20141230T235959UTC.csv', '7178I_20141231T000000UTC_20191230T235959UTC.csv', '7178I_20191231T000000UTC_20230101T235959UTC.csv']


### Utilización del módulo aemet_open_data desde la línea de comandos
Una vez que nos hemos familiarizado con el módulo, se puede ejecutar también desde la línea de comandos.<br>
Edita el fichero aemet_open_data_parameters.py con un editor de texto plano como NotePad, asigna los valores que deseas a los parámetros y ejecuta el fichero main.py.